In [ ]:
import random, os
import numpy as np
import torch

def seed_everything(seed: int):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(567)

: 

In [ ]:
/usr/bin/python3.8 -m pip install ipykernel -U --user --force-reinstall

Imports

In [25]:
from transformers import AutoTokenizer, AutoConfig, XLMRobertaAdapterModel
from datasets import load_dataset
from torch.utils.data import Dataset
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch import nn
import copy
from transformers import AdapterConfig
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction
from datasets import load_metric
import numpy as np
from transformers.adapters.composition import Stack
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

RuntimeError: Failed to import transformers.training_args because of the following error (look up to see its traceback):
No module named 'distutils.spawn'

Model

In [ ]:
#The labels for the NER task and the dictionaries to map the to ids or 
#the other way around
labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
id_2_label = {id_: label for id_, label in enumerate(labels)}
label_2_id = {label: id_ for id_, label in enumerate(labels)}

model_name = "xlm-roberta-base"
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels), label2id=label_2_id, id2label=id_2_label)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = XLMRobertaAdapterModel.from_pretrained(model_name, config=config)

In [ ]:
model.load_adapter_projections(['de', 'hi', 'is', 'es', 'id', 'ja', 'ta', 'th'], 0.9, '/content/subspace_cache')

In [ ]:
src_lang = "en" 
tgt_lang = "ja" 

# Load the language adapters
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
model.load_adapter(src_lang+"/wiki@ukp", config=lang_adapter_config) # leave_out=[11])
model.load_adapter(tgt_lang+"/wiki@ukp", config=lang_adapter_config) # leave_out=[11])

# Add a new task adapter
model.add_adapter("wikiann")

model.add_tagging_head("wikiann", num_labels=len(labels))

In [ ]:
model.train_adapter(["wikiann"])

Dataset

In [ ]:
dataset_src = load_dataset('wikiann', src_lang)
dataset_tgt = load_dataset('wikiann', tgt_lang)

# This method is adapted from the huggingface transformers run_ner.py example script
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    text_column_name = "tokens"
    label_column_name = "ner_tags"
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=False,
        truncation=True,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.  
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

  
dataset_src = dataset_src.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_src["train"].column_names,
)
dataset_tgt = dataset_tgt.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_tgt["train"].column_names,
)

data_collator = DataCollatorForTokenClassification(tokenizer,)

Training

In [ ]:
train_dataloader = DataLoader(
    dataset_src["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=16,
)
eval_dataloader = DataLoader(
    dataset_tgt["validation"], collate_fn=data_collator, batch_size=16
)

test_dataloader = DataLoader(
    dataset_tgt["test"], collate_fn=data_collator, batch_size=16
)

optimizer = AdamW(model.parameters(), lr=1e-4)

num_train_epochs = 15
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()
    label_list = id_2_label

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return true_labels, true_predictions

metric = load_metric("seqeval")

In [ ]:
progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda")
model.to(device)

for epoch in range(num_train_epochs):

    # Unfreeze and activate stack setup
    model.active_adapters = Stack(src_lang, "wikiann")
    model.activate_adapter_projection_stack('wikiann', 6, 'ja', 0.5)

    val_loss = 0
    # Training
    model.train()
    for batch in train_dataloader:
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


    # Evaluation
    model.eval()
    model.disable_adapter_projection_stack()
    model.active_adapters = Stack(tgt_lang, "wikiann")
    for batch in eval_dataloader:
        with torch.no_grad():
          inputs = batch['input_ids'].to(device)
          labels = batch['labels'].to(device)
          outputs = model(input_ids=inputs, labels=labels)
          val_loss += outputs.loss.item()

        predictions = outputs.logits.argmax(dim=2)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        #predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        #labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        #predictions_gathered = accelerator.gather(predictions)
        #labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions, labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )
    print(val_loss/len(eval_dataloader))

In [ ]:
test_loss = 0
for batch in test_dataloader:
    with torch.no_grad():
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs, labels=labels)
        test_loss += outputs.loss.item()

    predictions = outputs.logits.argmax(dim=2)
    labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        #predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        #labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        #predictions_gathered = accelerator.gather(predictions)
        #labels_gathered = accelerator.gather(labels)

    true_predictions, true_labels = postprocess(predictions, labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)

results = metric.compute()
print(f"epoch {epoch}:", {key: results[f"overall_{key}"] for key in ["precision", "recall", "f1", "accuracy"]},)
print(test_loss/len(test_dataloader))